In [1]:
from pyspark import SparkContext, SparkConf
conf = SparkConf().setAppName("Movies")
sc = SparkContext(conf = conf) 

In [2]:
df_rating = sc.textFile('./moviedata/rating.csv')
df_movies = sc.textFile('./moviedata/movies.csv')

In [3]:
def clean_df_movies(x):
    y = x.split('\"')
    return y[0].split(',')[0:1] + [y[1], ] +  y[2].split(',')[2:]

In [4]:
#Get RDD of all movies
all_movieids = df_movies.map(clean_df_movies).map(lambda x: (x[0], 0))

In [5]:
#Get RDD of rated movies
movieid_rating = df_rating.map(lambda x: x.split(",")).map(lambda x:(x[1], 1)).reduceByKey(lambda x,y: 1)

In [6]:
allmovies_ratedmovies = all_movieids.union(movieid_rating)

In [7]:
unrated_movies = allmovies_ratedmovies.reduceByKey(lambda x, y: x+y).filter(lambda x : x[1]==0)

In [8]:
unrated_movies = unrated_movies.map(lambda x: x[0])

In [9]:
unrated_movies.collect()

[u'1667',
 u'1663',
 u'1670',
 u'1666',
 u'1309',
 u'1669',
 u'1668',
 u'1310',
 u'1682']

In [10]:
#count of unrated movies, just for verification
all_movieids.count() - movieid_rating.count()

9